In [1]:
# https://www.kaggle.com/airbnb/seattle
# 上のサイトからダウンロードしたデータセット

In [2]:
import datetime
import graphviz
import itertools
import os
import platform
import socket
import sys
import time
import warnings
import category_encoders as ce
import lightgbm as lgb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pandas_profiling as pdp
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tqdm import tqdm
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 300)
sys.path.insert(0, '../utils/')
from utils import *

In [3]:
start("airbnb_seattle_cal")

DATE: 2019-04-04 17:37:31.955838
FILE: airbnb_seattle_cal
PID: 16072
HOST: Y-project-11
ENV: Windows-10-10.0.17134-SP0


In [4]:
file_name = "../data/input/airbnb_seattle/calendar.csv"
dtypes = {
    "listing_id" : "category",
    "date"       : "category",
    "available"  : "category",
    "price"      : "object"
}
with timer("read"):
    df = pd.read_csv(file_name, header=0, index_col=None, sep=',', dtype=dtypes)

[read] start
[read] done in 0 s


In [5]:
# 前処理
# date 列を datetime 型に変更
df["date"] = pd.to_datetime(df["date"])
# price 列を文字列処理
musk = ~df["price"].isnull()
df["price"][musk] = df["price"][musk].str.replace("$","")
df["price"][musk] = df["price"][musk].str.replace(",","")
df["price"] = df["price"].astype("float32")

In [6]:
df.shape

(1393570, 4)

In [7]:
df.head(3)

,listing_id,date,available,price
0,241032,2016-01-04,t,85.0
1,241032,2016-01-05,t,85.0
2,241032,2016-01-06,f,NaN


In [8]:
df.dtypes

listing_id          category
date          datetime64[ns]
available           category
price                float32
dtype: object

In [9]:
pdp.ProfileReport(df)

Number of variables,4
Number of observations,1393570
Total Missing (%),8.2%
Total size in memory,20.1 MiB
Average record size in memory,15.1 B
Numeric,1
Categorical,2
Date,1
Text (Unique),0
Rejected,0
Distinct count,2


- airbnb_seatle_calendar データ
- 期間：2016年1月4日～2017年1月2日（365日間）
- 対象施設：3818件
- 施設ごと日付ごとの予約可能の可否
- 予約可能の場合は価格